# 1.LCELでシンプルなテキスト生成アプリケーション

### (1)シックレートキー読み込み

In [ ]:
!pip install google-cloud-secret-manager

In [ ]:
from google.cloud import secretmanager
def access_secret_version(secret_id, version_id="latest"):
    client = secretmanager.SecretManagerServiceClient()
    name = client.secret_version_path("[Google Cloud Your Project]",secret_id, version_id)
    response = client.access_secret_version(request={"name": name})
    return response.payload.data.decode("UTF-8")

secret_id = "GEMINI_API_KEY"
api_key = access_secret_version(secret_id)

### (2)ライブラリインストール

In [1]:
# Vertex AI LLM SDK、langchainと依存関係をインストール
! pip install google-cloud-aiplatform langchain==0.0.323

  Using cached langchain-0.0.323-py3-none-any.whl.metadata (15 kB)
  Using cached langsmith-0.0.92-py3-none-any.whl.metadata (9.9 kB)
Using cached langchain-0.0.323-py3-none-any.whl (1.9 MB)
Using cached langsmith-0.0.92-py3-none-any.whl (56 kB)
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.1.117
    Uninstalling langsmith-0.1.117:
      Successfully uninstalled langsmith-0.1.117
  Attempting uninstall: langchain
    Found existing installation: langchain 0.2.16
    Uninstalling langchain-0.2.16:
      Successfully uninstalled langchain-0.2.16
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.2.16 requires langchain<0.3.0,>=0.2.16, but you have langchain 0.0.323 which is incompatible.
langchain-community 0.2.16 requires langsmith<0.2.0,>=0.1.0, but you have langsmith 0.0.92 which is incompatible.
langchain-co

## (3)ライブラリのバージョン表示

In [2]:
# Utils ユーティリティ関数
from langchain.schema import HumanMessage, SystemMessage  # LangChainのHumanMessageとSystemMessageをインポートします。
from langchain.llms import VertexAI                       # LangChainのVertexAIをインポートします。
from langchain.embeddings import VertexAIEmbeddings       # LangChainのVertexAIEmbeddingsをインポートします。
from langchain.chat_models import ChatVertexAI            # LangChainのChatVertexAIをインポートします。
from google.cloud import aiplatform                       # google-cloud-aiplatformからaiplatformをインポートします。
import time                                               # timeモジュールをインポートします。
from typing import List                                   # typingモジュールからListをインポートします。

# LangChain
import langchain
from pydantic import BaseModel

print(f"LangChain version: {langchain.__version__}")  # LangChainのバージョンを表示します。

# Vertex AI
print(f"Vertex AI SDK version: {aiplatform.__version__}") # Vertex AI SDKのバージョンを表示します。

LangChain version: 0.0.323
Vertex AI SDK version: 1.66.0


### (4)モデル初期化

In [3]:
import os

# os.environ["GOOGLE_API_KEY"] = secret_id
from langchain_google_vertexai import ChatVertexAI

model = ChatVertexAI(model="gemini-1.5-flash")

### (5)LLMモデルのシンプルな生成

プロンプトテンプレート、Outputパーサー設定

In [4]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("{topic}について、1行のビジネス川柳を作成してね")

chain = prompt | model | StrOutputParser()

プロンプトから生成を実行

In [5]:
chain.invoke({"topic": "LLM"})

'言葉紡ぎ、知恵と創造、未来拓く。 \n'

In [6]:
chain.invoke({"topic": "Google Cloud"})

'クラウドの力、Googleが支え、ビジネス躍進！ \n'



---



# 2.チャットボット

### (1)モデル初期化

In [7]:
from langchain_google_vertexai import ChatVertexAI

model = ChatVertexAI(model="gemini-1.5-flash")

### (2)Chatモデルのシンプルな生成

In [8]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="こんにちは、ボブ！")])

AIMessage(content='こんにちは！ 何かお手伝いできることはありますか？ 😄\n', response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}], 'usage_metadata': {'prompt_token_count': 5, 'candidates_token_count': 13, 'total_token_count': 18, 'cached_content_token_count': 0}, 'finish_reason': 'STOP'}, id='run-fc3f69ae-5494-4c9e-990f-b9aa04638465-0', usage_metadata={'input_tokens': 5, 'output_tokens': 13, 'total_tokens': 18})

In [9]:
model.invoke([HumanMessage(content="私の名前は?")])

AIMessage(content='私は、あなたに関する情報を保存していません。あなたに名前を教えてください。 😊 \n', response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}], 'usage_metadata': {'prompt_token_count': 4, 'candidates_token_count': 18, 'total_token_count': 22, 'cached_content_token_count': 0}, 'finish_reason': 'STOP'}, id='run-091cfa02-d05b-4d12-a165-2f3e538d194a-0', usage_metadata={'input_tokens': 4, 'output_tokens': 18, 'total_tokens': 22})

In [10]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="こんにちは、私はボブです。"),
        AIMessage(content="こんにちは、ボブ! ご機嫌はいかが?"),
        HumanMessage(content="私の名前は?"),
    ]
)

AIMessage(content='あなたの名前はボブですよ！😊 \n\n何か私に聞いてみたいことはありますか？ \n', response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}], 'usage_metadata': {'prompt_token_count': 22, 'candidates_token_count': 19, 'total_token_count': 41, 'cached_content_token_count': 0}, 'finish_reason': 'STOP'}, id='run-a09bd904-5c82-4dfd-a873-d1f25f843736-0', usage_metadata={'input_tokens': 22, 'output_tokens': 19, 'total_tokens': 41})

### (3)会話履歴をメモリに保存し、Chat生成

Message History クラスを使用してモデルをラップし、ステートフルにすることができます。

In [11]:
!pip install langchain_community

  Using cached langchain-0.2.16-py3-none-any.whl.metadata (7.1 kB)
  Using cached langsmith-0.1.117-py3-none-any.whl.metadata (13 kB)
Using cached langchain-0.2.16-py3-none-any.whl (1.0 MB)
Using cached langsmith-0.1.117-py3-none-any.whl (290 kB)
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.0.92
    Uninstalling langsmith-0.0.92:
      Successfully uninstalled langsmith-0.0.92
  Attempting uninstall: langchain
    Found existing installation: langchain 0.0.323
    Uninstalling langchain-0.0.323:
      Successfully uninstalled langchain-0.0.323


関連するクラスをインポートし、モデルをラップしてこのメ​​ッセージ履歴を追加するチェーンを設定します。

get_session_history関数は、 session_idを受け取り、メッセージ履歴オブジェクトを返すことが期待されます。

In [12]:
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

configここで、毎回実行可能ファイルに渡す を作成する必要があります。

In [13]:
config = {"configurable": {"session_id": "abc2"}}

In [14]:
response = with_message_history.invoke(
    [HumanMessage(content="私は、ボブです。")],
    config=config,
)

response.content

'はじめまして、ボブさん。どうぞよろしくお願いします。😊  何か私にできることはありますか？ \n'

In [15]:
response = with_message_history.invoke(
    [HumanMessage(content="私の名前は?")],
    config=config,
)

response.content

'あなたの名前はボブです！  😉  私があなたを覚えていますよ。 \n'

# 3.テキスト エンベディングを取得

In [16]:
from vertexai.language_models import TextEmbeddingModel

def get_vertexai_embeddings(texts):
    # Initialize the Vertex AI Text Embedding Model
    model = TextEmbeddingModel.from_pretrained("textembedding-gecko")

    # Generate embeddings for the provided texts
    embeddings = model.get_embeddings(texts)

    return embeddings

# Sample texts
texts = ["Hello, world!", "This is a test sentence for generating embeddings."]
embeddings = get_vertexai_embeddings(texts)

# Print the embeddings
for text, embedding in zip(texts, embeddings):
    print(f"Text: {text}")
    print(f"Embedding: {embedding}")
    print("\n")

Text: Hello, world!
Embedding: TextEmbedding(values=[0.05048881471157074, -0.027526220306754112, -0.029739122837781906, -0.023983176797628403, 0.014556266367435455, 0.009877710603177547, 0.031096046790480614, -0.013410440646111965, 0.010270212776958942, -0.0028309847693890333, 0.04382384195923805, 0.04735852777957916, 0.0038899395149201155, -0.05870475620031357, -0.013154584914445877, -0.018172577023506165, 0.01607293263077736, -0.008533775806427002, -0.004098878242075443, -0.004364946857094765, 0.003848735708743334, 0.004871689714491367, -0.023533562198281288, -0.03507405146956444, 0.02098313719034195, -0.0014545134035870433, 0.0008274627616629004, -0.07081551104784012, 0.0011253736447542906, 0.06359116733074188, -0.04751729965209961, 0.007458766922354698, -0.05418795347213745, 0.008057346567511559, 0.040036819875240326, -0.06546219438314438, 0.040994979441165924, 0.011809512041509151, -0.0030763561371713877, 0.006217926740646362, 0.02165478654205799, -0.08145613968372345, -0.04319110